In [7]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import window, col,  explode, split, concat, col, lit
from pyspark.sql.types import StructType, StructField, LongType, StringType, DoubleType
from time import sleep


sparkConf = SparkConf()
sparkConf.setMaster("spark://spark-master:7077")
sparkConf.setAppName("Streaming_pipeline")
sparkConf.set("spark.driver.memory", "2g")
sparkConf.set("spark.executor.cores", "1")
sparkConf.set("spark.driver.cores", "1")

# create the spark session, which is the entry point to Spark SQL engine.
spark = SparkSession.builder.config(conf=sparkConf).getOrCreate()

# Read the whole dataset as a batch
df = spark \
        .readStream \
        .format("kafka") \
        .option("kafka.bootstrap.servers", "kafka1:9093").option("failOnDataLoss", "false") \
        .option("subscribe", "games_details") \
        .option("startingOffsets", "earliest") \
        .load()

lines = df.selectExpr("CAST(value AS STRING)")

words = lines.select(
        explode(
            split(lines.value, " ")
        ).alias("word")
    )

wordCounts = words.groupBy("word").count()

query = wordCounts \
              .select(concat(col("word"), lit(" "), col("count")).alias("value")) \
              .writeStream \
              .format("kafka") \
              .option("kafka.bootstrap.servers", "kafka1:9093").option("failOnDataLoss", "false") \
              .option("checkpointLocation", "/home/jovyan/checkpoint/games_details")\
              .option("topic", "games_details") \
              .outputMode("complete") \
              .start()

try:
    query.awaitTermination()
except KeyboardInterrupt:
    query.stop()
    # Stop the spark context
    spark.stop()
    print("Stoped the streaming query and the spark context")
except:
    query.stop()
    # Stop the spark context
    spark.stop()
    print("Unexpected error")
    print("Stoped the streaming query and the spark context")


# Define the schema for games_details.json
# gamesDetailsSchema = StructType([
#     StructField("game_id", StringType(), True),
#     StructField("player_id", StringType(), True),
#     StructField("team_id", StringType(), True),
#     StructField("player_name", StringType(), True),
#     StructField("team_abbreviation", StringType(), True),
#     StructField("min", StringType(), True),
#     StructField("ast", IntegerType(), True),
#     StructField("stl", IntegerType(), True),
#     StructField("pf", IntegerType(), True)
# ])

# # Read from a source 
# sdf = spark.readStream.schema(dataSchema).option("maxFilesPerTrigger", 1) \
#         .json("/home/jovyan/data/games_details")

# # Write to a sink - here, the output is memory (only for testing). The query name (i.e., activity_counts) will be the Spark SQL table name.
# gamesDetailsQuery = activityCounts.writeStream.queryName("games_details_query") \
#                     .format("memory").outputMode("complete") \
#                     .start()
# # Testing 
# for x in range(10):
#     spark.sql("SELECT * FROM games_details_query").show()
#     sleep(5)

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
                          ^^^^^^^^^^^^^^^^^^^^^^
RuntimeError: reentrant call inside <_io.BufferedReader name=51>

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 539, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while sending or receiving
ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/usr/local/spark/python/lib/py4j-0.10.9.7-src.zip

Unexpected error
Stoped the streaming query and the spark context


In [8]:
spark.stop()